In [1]:
import os
import pandas as pd
import random
import ast

In [ ]:
def shuffle_choices(row):
    """
    Shuffle the answer choices for a given row.
    """
    choices = ast.literal_eval(row['candidates'])
    answer = row['answer']

    correct_choice = [choice for choice in choices if choice.startswith(answer)][0]
    answer_text = correct_choice.split('.')[1].strip()
    
    choice_texts = [choice.split('.')[1].strip() for choice in choices]
    random.shuffle(choice_texts)

    letters = ['A', 'B', 'C']
    shuffled_choices = [f"{letters[i]}. {choice}" for i, choice in enumerate(choice_texts)]
    correct_index = choice_texts.index(answer_text)
    correct_letter = letters[correct_index]

    return pd.Series({
        'candidates': shuffled_choices,
        'answer': correct_letter
    })
    

In [3]:
file_path = os.path.expanduser('~/LMUData') 
MCQ_path = os.path.join(file_path, 'Virat_MCQ_frames.tsv')
MCQ_shuffled_path = os.path.join(file_path, 'Virat_MCQ_frames_shuffled.tsv')
MCQ = pd.read_csv(MCQ_path, sep='\t')

MCQ[['candidates', 'answer']] = MCQ.apply(shuffle_choices, axis=1)
MCQ.to_csv(MCQ_shuffled_path, sep='\t', index=False)

In [73]:
import json

file_path = os.path.expanduser('~/LMUData') 
# json_path = os.path.join(file_path, 'Virat_MCQ_train_text.json')
# json_shuffled_path = os.path.join(file_path, 'Virat_MCQ_train_text_shuffled.json')
json_path = os.path.join(file_path, 'Virat_MCQ_validation_text.json')
json_shuffled_path = os.path.join(file_path, 'Virat_MCQ_validation_text_shuffled.json')


with open(json_path, 'r', encoding='utf-8') as f:
    MCQJ = json.load(f)

abc = [*'ABC']
for Q in MCQJ:
    Us,As = Q["messages"]
    # print(Us['content'])
    content, choice_texts = Us['content'].split('[')
    choice_texts = list(map(lambda x: x.strip(" '"), choice_texts[:-1].split(',')))
    random.shuffle(choice_texts)
    assert As['content'] in choice_texts, (As['content'],choice_texts)
    for i in range(len(choice_texts)):
        if As['content'] == choice_texts[i]:
            As['content'] = abc[i] + As['content'][1:]
        choice_texts[i] = abc[i] + choice_texts[i][1:]
    Us['content'] = content + str(choice_texts)
    # print(Us['content'])
    # print(As['content'])

with open(json_shuffled_path, 'w', encoding='utf-8') as f:
    json.dump(MCQJ, f, indent=2)